In [20]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [23]:
df = pd.read_excel('../data/Becker_promoter/1-s2.0-S1525001625003016-mmc2.xlsx')
df = df.iloc[1:, 1:]
df.columns = df.iloc[0]
df = df.iloc[1:, :].reset_index(drop=True)
df = df.rename(columns={'sequence': 'seq'})
df

1,Promoter name,Description,Supposed specificity,Origin (viral/mammalian/synthetic/chimeric),seq,Total size of promoter (bp),Reference (PubMed ID),Size of LacZ stuffer (bp),Final size of the construct
0,AAT,Promoter of Homo sapiens serpin family A membe...,liver,mammalian,gatcttgctaccagtggaacagccactaaggattctgcagtgagag...,397,10933977,2000,3923
1,AAT short,Short version of hAAT promoter,liver,mammalian,aatgactcctttcggtaagtgcagtggaagctgtacactgcccagg...,251,10933977,2200,3977
2,ADIPOQ,"Human Adiponectin, C1Q and collagen domain con...",fat,mammalian,gcccttcaaaaacaaaacatgagcgtgccaagaaagtccaaggtgt...,1077,15850785; 18931025,1500,4103
3,Albumin,Mus musculus albumin (Alb) 5' regulatory regio...,liver,mammalian,ggcatgcttccatgccaaggcccacactgaaatgctcaaatgggag...,335,2722822,2200,4061
4,AP2.2,Promoter region of Mus musculus adiponectin (r...,fat,mammalian,aaaccacccagcaaaaaaccaaaccgcctagcctcaagacatgtgt...,2274,18760339; 25551639,no LacZ,3800
...,...,...,...,...,...,...,...,...,...
53,TnT,"Promoter of Homo sapiens troponin T2, cardiac ...",heart,mammalian,ctcagtccattaggagccagtagcctggaagatgtctttaccccca...,544,25082846,2000,4070
54,TTR,Enhancer + promoter region of Mus musculus tra...,liver,mammalian,ggatctgtcaattcacgcgagttaataattaccagcgcgggccaaa...,541,1690125,2000,4067
55,UBC,Promoter of Homo sapiens ubiquitin C (UBC) (re...,ubiquitous,mammalian,ggtgcagcggcctccgcgccgggttttggcgcctcccgcgggcgcc...,1258,11704814,1200,3984
56,UCP1,Mus musculus uncoupling protein 1 (Ucp) distal...,mitochondria/brown adipose tissue,mammalian,cgtcacagagggtcagtcacccttgaccacactgaactagtcgtca...,527,28930673; 8264627,2000,4053


In [24]:
df.to_csv('../data/Becker_promoter/Becker_promoter_long.tsv', sep='\t', index=False)

In [25]:
from MPRA_predict import models, datasets, metrics, utils
from MPRA_predict.utils import *

data_path = '../data/Becker_promoter/Becker_promoter_long.tsv'
device = 'cuda:1'
output_path = 'outputs/Becker_promoter_long_Sei_pred.h5'

In [26]:
@torch.no_grad()
def get_pred(model, test_data_loader, device='cuda', writer: HDF5Writer=None, flush_every=16):
    model = model.to(device)
    y_pred = []
    model.eval()
    for i, batch in enumerate(tqdm(test_data_loader)):
        if isinstance(batch, (list, tuple)):
            x = batch[0]
        elif isinstance(batch, dict):
            x = batch['seq']
        else:
            x = batch
        x = x.to(device)
        output = model(x)

        output = output.detach().cpu().numpy()
        writer.append(output)
        if (i+1) % flush_every == 0:
            writer.flush()
    writer.flush()
    writer.close()




model_path = f'../data/Sei/resources/sei.pth'
model_state_dict = torch.load(model_path)
model_state_dict = {k.replace('module.model.', ''): v for k, v in model_state_dict.items()}
model = models.Sei()
model.load_state_dict(model_state_dict, strict=False)
model = model.to(device)

dataset = datasets.SeqDataset(
    data_path=data_path,
    seq_column='seq', 

    crop=False,
    # crop=True,
    # cropped_length=200,
    padding=True,
    padding_method='N',
    padded_length=4096,
    N_fill_value=0.25,
)

test_data_loader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)
writer = HDF5Writer(output_path)
pred = get_pred(model, test_data_loader, device, writer)

100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


In [28]:
pred_array = load_h5('outputs/Becker_promoter_long_Sei_pred.h5')
pred_array.shape

file outputs/Becker_promoter_long_Sei_pred.h5 has keys: ['data']


(58, 21907)